## Import libraries

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score
import gc

## Read heldout processed data

In [4]:
heldout_df = pd.read_csv('/Users/adityaponnada/Downloads/time_study_data/general_heldout_scaled.csv')  # Replace with actual path
heldout_df.head()

,participant_id,outcome,is_weekend,in_battery_saver_mode,charging_status,screen_on,dist_from_home,is_phone_locked,last_phone_usage,closeness_to_sleep_time,...,wake_day_part_2.0,wake_day_part_3.0,mi_in_battery_saver_mode,mi_charging_status,mi_dist_from_home,mi_is_phone_locked,mi_last_phone_usage,mi_closeness_to_sleep_time,mi_closeness_to_wake_time,mi_mims_5min
0,animateshowerclothes@timestudy_com,1,0,1.0,0.0,1,-40.137687,1.0,-0.589123,1.842273,...,0,0,0,0,0,1,1,0,0,0
1,animateshowerclothes@timestudy_com,0,0,1.0,1.0,1,-40.136734,1.0,-0.589123,1.739076,...,0,0,0,0,0,1,1,0,0,0
2,animateshowerclothes@timestudy_com,0,0,0.0,1.0,1,-40.137375,1.0,-0.589123,1.694101,...,0,0,0,0,0,1,1,0,0,0
3,animateshowerclothes@timestudy_com,0,0,0.0,1.0,1,-40.133405,1.0,-0.589123,1.649200,...,0,0,0,0,0,1,1,0,0,0
4,animateshowerclothes@timestudy_com,0,0,0.0,1.0,1,-40.132982,1.0,-0.589123,1.487557,...,0,0,0,0,0,1,1,0,0,0


In [5]:
def max_observations_per_participant(df, participant_col='participant_id'):
    """Return the maximum number of observations any participant has in `df`."""
    import pandas as pd
    if df is None or participant_col not in df.columns:
        raise ValueError('DataFrame must contain the participant column')
    counts = df[participant_col].value_counts()
    if counts.empty:
        return 0
    return int(counts.max())

# Example usage
max_obs = max_observations_per_participant(heldout_df)
print('max observations per participant:', max_obs)
max_obs

max observations per participant: 15169


15169

In [6]:
# Print number of columns excluding participant_id and outcome
excluded = {'participant_id', 'outcome'}
feature_cols = [c for c in heldout_df.columns if c not in excluded]
print('number of feature columns (excluding participant_id and outcome):', len(feature_cols))
# optional: show first few feature column names
print('example feature columns:', feature_cols[:10])

number of feature columns (excluding participant_id and outcome): 40
example feature columns: ['is_weekend', 'in_battery_saver_mode', 'charging_status', 'screen_on', 'dist_from_home', 'is_phone_locked', 'last_phone_usage', 'closeness_to_sleep_time', 'closeness_to_wake_time', 'mims_5min']
